In [ ]:
# prompt: export pretrained keras mobilenet model to tflite, quantize to INT8

import tensorflow as tf

# download the weights from
# https://github.com/NavodPeiris/MobileNet_96x96_greyscale_weights
!wget https://raw.githubusercontent.com/NavodPeiris/MobileNet_96x96_greyscale_weights/main/mobilenetV1_0.1_96x96_greyscale_weights.h5
!wget https://raw.githubusercontent.com/NavodPeiris/MobileNet_96x96_greyscale_weights/main/mobilenetV1_0.2_96x96_greyscale_weights.h5
!wget https://raw.githubusercontent.com/NavodPeiris/MobileNet_96x96_greyscale_weights/main/mobilenetV1_0.25_96x96_greyscale_weights.h5

In [ ]:
# download cat vs. dog dataset
# from https://www.kaggle.com/datasets/anthonytherrien/dog-vs-cat

from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"
!kaggle datasets download -d anthonytherrien/dog-vs-cat
!unzip -q dog-vs-cat.zip -d .
!ls

In [ ]:
IMAGE_SIZE = 96
BATCH_SIZE = 32
EPOCHS = 100
COLOR_MODE = "grayscale"

dataset = tf.keras.preprocessing.image_dataset_from_directory("animal",
                                                             shuffle = True,
                                                             image_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                              color_mode = COLOR_MODE,
                                                              batch_size = BATCH_SIZE,
                                                              label_mode = "categorical"
                                                             )
def get_data_splitting_tf(dataset,
                      train_split = 0.7,
                      test_split = 0.15,
                      val_split = 0.15,
                      shuffle = True, # data will be shuffle show that no particular image in sellected more than once durring the splitting
                      shuffle_size = 100000 #
                      ):


    data_size = len(dataset)
    if shuffle:

        dataset = dataset.shuffle(shuffle_size, seed = 42)
    train_size = int(train_split * data_size)

    test_size = int(test_split * data_size)

    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size).take(test_size) # the test data will skip the data in the train dataset

    val_dataset = dataset.skip(train_size).skip(test_size) # this will also skip all the dataset that has already been selected by the test and train size

    return train_dataset, test_dataset, val_dataset

train_dataset , test_dataset , val_dataset = get_data_splitting_tf(dataset)

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size =  tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

rescale_and_resize = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1.0/255)
]) # Rescale and resizing the data

# Data augmentation is done toe to introduce the data to different orrientations and also increase the size of the dataset

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("Horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.3)
    ], name="data_augmentation")

In [ ]:
class_names = dataset.class_names

import matplotlib.pyplot as plt

plt.figure(figsize = (15, 15))

for image_batch, label_batch in dataset.take(1):
    for j in range(20): # Showing 20 images from among the dadtaset in the two classes
        ax = plt.subplot(4,5, j+1)
        plt.imshow(image_batch[j].numpy().astype("uint8"))
        plt.title(class_names[label_batch[j].numpy().argmax()])

        plt.axis(False) # do not show axis for the images

In [ ]:
# clear keras session
tf.keras.backend.clear_session()

input_tensor = tf.keras.layers.Input(shape=(96, 96, 1))
aug = data_augmentation(input_tensor)

mobilenet_model = tf.keras.applications.MobileNet(
    input_shape=(96, 96, 1),
    input_tensor=aug,
    pooling="avg",
    alpha=0.1,   # 0.25, 0.2, 0.1
    weights="mobilenetV1_0.1_96x96_greyscale_weights.h5", # 0.25, 0.2, 0.1
    include_top=False
    )

mobilenet_model.trainable = False

mobilenet_output = mobilenet_model.output

# classification layer
classification_layer = tf.keras.layers.Dense(2, activation='softmax')(mobilenet_output)

model = tf.keras.Model(inputs=mobilenet_model.input, outputs=classification_layer)

print("Compiling model...")
model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=["accuracy"])

model.summary()

In [ ]:
history = model.fit(
                   train_dataset,
                   epochs = EPOCHS,
                   batch_size = BATCH_SIZE,
                   verbose = 1,
                   validation_data = val_dataset
                   )

In [ ]:
def representative_data_gen():
  for image_batch, label_batch in dataset.take(1):
    yield [image_batch]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to int8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()
# Save the quantized model
with open('model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [ ]:
import numpy as np

# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_images):

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_images),), dtype=int)
  for i, test_image in enumerate(test_images):

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.int8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions

# Change this to test a different image
test_image_index = np.random.randint(0, BATCH_SIZE)

## Helper function to test the models on one image
def test_model(tflite_file, test_image_index):

  # take one image from test dataset
  test_data = list(test_dataset.take(2).as_numpy_iterator())[0]
  test_image = test_data[0][test_image_index]
  test_label = test_data[1][test_image_index]
  predictions = run_tflite_model(tflite_file, [test_image])
  plt.imshow(test_image)
  template = " Model \n True:{true}, Predicted:{predict}"
  _ = plt.title(template.format(true = str(test_label.argmax()), predict=str(predictions[0])))
  plt.grid(False)

test_model('model_quant.tflite', test_image_index)

In [ ]:
def evaluate_model(tflite_file):

  test_data = list(test_dataset.take(1).as_numpy_iterator())[0]
  test_images = test_data[0]
  test_labels = test_data[1].argmax(axis=1)
  predictions = run_tflite_model(tflite_file, test_images)

  accuracy = (np.sum(test_labels==predictions) * 100) / len(test_images)

  print('model accuracy is %.4f%% (Number of test samples=%d)' % (accuracy, len(test_images)))

evaluate_model('model_quant.tflite')

In [ ]:
!xxd -i  model_quant.tflite > model.h